In [1]:
import numpy as np
import cv2
import random

In [2]:
#Import cascade
face_cascade = cv2.CascadeClassifier('C:/Users/lenovo/Jupyter Projects/Mask and not mask/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('C:/Users/lenovo/Jupyter Projects/Mask and not mask/haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier('C:/Users/lenovo/Jupyter Projects/Mask and not mask/Mouth.xml')
upper_body = cv2.CascadeClassifier('C:/Users/lenovo/Jupyter Projects/Mask and not mask/haarcascade_upperbody.xml')

In [3]:
# Adjust threshold value in range 80 to 105 based on your light.
brightness_threshold = 100

In [4]:
# User message
font = cv2.FONT_HERSHEY_SCRIPT_COMPLEX
org = (30, 30)
FONT_MASK = (0, 255, 0)
FONT_NOT_MASK = (0, 0, 255)
FONT_SCALE = 1
MESSAGE_WEAR = "Mask Detected !!"
MESSAGE_NOT_WEAR = "Mask not Detected !!"

# get video input
video = cv2.VideoCapture(0)
while 1:
 
    ret, img = video.read()
    
    #Lets flip array of image ..  0 reprsnet x-axis and 1 represent y-axis
    img = cv2.flip(img,1)

    # changing Image into gray scale 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Convert image in black and white
    (thresh, black_and_white) = cv2.threshold(gray, brightness_threshold, 255, cv2.THRESH_BINARY)
  

  # detect face
    FACES = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Face prediction for black and white
    faces_bw = face_cascade.detectMultiScale(black_and_white, 1.1, 4)


    if(len(FACES) == 0 and len(faces_bw) == 1):
       # mask not found !!
        cv2.putText(img, MESSAGE_WEAR, org, font, FONT_SCALE, FONT_MASK, 2, cv2.LINE_AA)
    else:
        # Draw rectangle on face
        for (x, y, w, h) in FACES:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 255), 2)
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = img[y:y + h, x:x + w]


            # Detect lips counters
            RECTANGLE_MOUTH = mouth_cascade.detectMultiScale(gray, 1.5, 5)

        # Face detected but Lips not detected which means person is wearing mask
        if(len(RECTANGLE_MOUTH) == 0):
            cv2.putText(img, MESSAGE_WEAR, org, font, FONT_SCALE, FONT_MASK, 2, cv2.LINE_AA)
        else:
            for (m_x, m_y, m_w, m_h) in RECTANGLE_MOUTH:

                if(y < m_y < y + h):
                    # Face and Lips are detected but lips coordinates are within face cordinates which `means lips prediction is true and
                    # person is not waring mask
                    cv2.putText(img, MESSAGE_NOT_WEAR, org, font, FONT_SCALE, FONT_NOT_MASK, 2, cv2.LINE_AA)
                    break

    # Show frame with results
    cv2.imshow('Mask Detection', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

# Release video
video.release()
cv2.destroyAllWindows()